In [1]:
!pip install -qqq "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" --progress-bar off
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install -qqq --no-deps {xformers} trl peft accelerate bitsandbytes triton --progress-bar off

import torch
from trl import SFTTrainer
from datasets import load_dataset
from transformers import TrainingArguments, TextStreamer
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel, is_bfloat16_supported

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
grpcio-status 1.62.3 requires protobuf>=4.21.6, but you have protobuf 3.20.3 which is incompatible.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


# 1. Fine-Tune Llama-3.1-8B

In [2]:
max_seq_length = 8000
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="fulim/FineLlama-3.1-8B",
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    dtype=None,
)

==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
alpaca_prompt = """You are a storywriter, Complete the instruction
### Instruction:
Generate a story based on the provided genre and title.
Ensure the output includes the characters and the story.

### Input:
Genre: {}
Title: {}

### Response:
Story: {}
Characters: {}
"""

# Ensure EOS_TOKEN is set to a valid string, either from tokenizer or default it to a custom string
EOS_TOKEN = tokenizer.eos_token if tokenizer.eos_token is not None else "<|endoftext|>"

# Print EOS_TOKEN to debug
print(f"EOS_TOKEN: {EOS_TOKEN}")

FastLanguageModel.for_inference(model)
text_streamer = TextStreamer(tokenizer)

inputs = {
    "genre": "Fantasy",
    "title": "Write a Story about James and Alice Adventure"
}

# Assuming inputs contain genre and title
genre = inputs["genre"]
title = inputs["title"]

# Construct the alpaca prompt
formatted_prompt = alpaca_prompt.format(genre, title, "" , "")

# Add EOS token to the formatted prompt
formatted_prompt_with_eos = formatted_prompt

# Tokenize the formatted prompt with EOS token
input_ids = tokenizer(formatted_prompt_with_eos, return_tensors="pt").input_ids

# Generate text using the model
_ = model.generate(input_ids=input_ids, streamer=text_streamer, max_new_tokens=2000)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


EOS_TOKEN: <|end_of_text|>
<|begin_of_text|>You are a storywriter, Complete the instruction
### Instruction:
Generate a story based on the provided genre and title.
Ensure the output includes the characters and the story.

### Input:
Genre: Fantasy
Title: Write a Story about James and Alice Adventure

### Response:
Story: 
Characters: 
James and Alice

Story:
Once upon a time, there lived a brave and adventurous princess named Alice. She was known for her quick thinking and ability to solve complex problems. One day, while exploring the forest, Alice stumbled upon a mysterious castle. Curiosity got the better of her, and she decided to enter the castle, despite the warnings from her father, the king. As she entered the castle, she was greeted by a tall and handsome prince named James. James was known for his kindness and compassion, and he immediately took a liking to Alice. He invited her to stay at the castle and promised to protect her from any harm.

As the days went by, Alice and 

# Separate as two separate Notebook

# 2. Base-Model Llama-3.1-8B

In [5]:
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
max_seq_length = 8000
base_model, base_tokenizer = FastLanguageModel.from_pretrained(
    model_name="meta-llama/Llama-3.1-8B",
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    dtype=None,
)

==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

meta-llama/Llama-3.1-8B does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.


In [9]:
from transformers import TextStreamer

In [3]:
alpaca_prompt = """You are a storywriter, Complete the instruction
### Instruction:
Generate a story based on the provided genre and title.
Ensure the output includes the characters and the story.

### Input:
Genre: {}
Title: {}

### Response:
Story: {}
Characters: {}
"""

# Ensure EOS_TOKEN is set to a valid string, either from tokenizer or default it to a custom string
EOS_TOKEN = base_tokenizer.eos_token if base_tokenizer.eos_token is not None else "<|endoftext|>"

# Print EOS_TOKEN to debug
print(f"EOS_TOKEN: {EOS_TOKEN}")

FastLanguageModel.for_inference(base_model)
text_streamer = TextStreamer(base_tokenizer)

inputs = {
    "genre": "Fantasy",
    "title": "Write a Story about James and Alice Adventure"
}

# Assuming inputs contain genre and title
genre = inputs["genre"]
title = inputs["title"]

# Construct the alpaca prompt
formatted_prompt = alpaca_prompt.format(genre, title, "" , "")

# Add EOS token to the formatted prompt
formatted_prompt_with_eos = formatted_prompt

# Tokenize the formatted prompt with EOS token
input_ids = base_tokenizer(formatted_prompt_with_eos, return_tensors="pt").input_ids

# Generate text using the model
_ = base_model.generate(input_ids=input_ids, streamer=text_streamer, max_new_tokens=2000)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


EOS_TOKEN: <|end_of_text|>
<|begin_of_text|>You are a storywriter, Complete the instruction
### Instruction:
Generate a story based on the provided genre and title.
Ensure the output includes the characters and the story.

### Input:
Genre: Fantasy
Title: Write a Story about James and Alice Adventure

### Response:
Story: 
Characters: 
1. James
2. Alice
Story: 
James and Alice are two friends who love to explore the world. One day, they decide to go on a journey to find the legendary treasure of the ancient kingdom. They set out on their adventure, equipped with their trusty map and compass. As they travel through the dense forest, they come across a group of bandits who try to steal their treasure. James and Alice manage to escape, but not before Alice is injured. They continue their journey, but the forest becomes increasingly dangerous, with wild animals and treacherous terrain. They eventually find the ancient kingdom, but the treasure is guarded by a fierce dragon. James and Alice